In [1]:
# LSTM for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)


In [2]:
#Refer: https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification

# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=top_words)


17473536/17464789 [==============================] - 0s 0us/step


In [3]:
print(X_train[1])
print(type(X_train[1]))
print(len(X_train[1]))

[1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369, 2, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 2, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 2, 5, 163, 11, 3215, 2, 4, 1153, 9, 194, 775, 7, 2, 2, 349, 2637, 148, 605, 2, 2, 15, 123, 125, 68, 2, 2, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 2, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 2, 5, 2, 656, 245, 2350, 5, 4, 2, 131, 152, 491, 18, 2, 32, 2, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95]
<class 'list'>
189


In [4]:
X_train.shape

(25000,)

In [5]:
max(numpy.max(X_test))

4998

In [6]:
# truncate and/or pad input sequences
max_review_length = 600
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

print(X_train.shape)
print(X_train[1])

(25000, 600)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    

In [7]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#Refer: https://datascience.stackexchange.com/questions/10615/number-of-parameters-in-an-lstm-model

# parameters = 4(nm+n^2+n)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 600, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#the LSTM has a set of 2 matrices: U and W for each of the (3) gates. The (.) 
# in the diagram indicates multiplication of these matrices with the input x and output h.

#U has dimensions n×m
#W has dimensions n×n
#there is a different set of these matrices for each of the three gates(like Uforget for the forget gate etc.)
#there is another set of these matrices for updating the cell state S
#on top of the mentioned matrices, you need to count the biases (not in the picture)
#Hence total # parameters = 4(nm+n^2+n)

In [ ]:
# Interview question :
# LSTM - 1997 : 3 GATE (Input , output and forget)

# GRU - Gated Recurrent Unit : 2014 -  2 Gate ( reset, update)
#--  Simplified version of LSTM - Inspired by LSTM and just enhancement of calculation 
#--  Faster to train
#--  This is as powerful as LSTM

# Note : There is no clear winner as which model (LSTM or GRU) is better. Till 2021, programmer/data scientist prefer to 
    # use LSTM as compare to new model call GRU

In [8]:
model.fit(X_train, y_train, epochs=20, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/20
391/391 [==============================] - 41s 90ms/step - loss: 0.4799 - accuracy: 0.7544
Epoch 2/20
391/391 [==============================] - 35s 90ms/step - loss: 0.2908 - accuracy: 0.8830
Epoch 3/20
391/391 [==============================] - 35s 90ms/step - loss: 0.2373 - accuracy: 0.9081
Epoch 4/20
391/391 [==============================] - 35s 90ms/step - loss: 0.2103 - accuracy: 0.9202
Epoch 5/20
391/391 [==============================] - 35s 90ms/step - loss: 0.1901 - accuracy: 0.9283
Epoch 6/20
391/391 [==============================] - 35s 90ms/step - loss: 0.3497 - accuracy: 0.8342
Epoch 7/20
391/391 [==============================] - 35s 90ms/step - loss: 0.5483 - accuracy: 0.7342
Epoch 8/20
391/391 [==============================] - 35s 90ms/step - loss: 0.4525 - accuracy: 0.8040
Epoch 9/20
391/391 [==============================] - 35s 90ms/step - loss: 0.2265 - accuracy: 0.9099
Epoch 10/20
391/391 [==============================] - 35s 90ms/step - loss: 0.181